<a href="https://colab.research.google.com/github/Ayodeji-python/SIT723/blob/main/User_Stories_Extraction_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install libraries
!pip install -U spacy

     |████████████████████████████████| 5.8 MB 13.7 MB/s 
     |████████████████████████████████| 10.1 MB 43.7 MB/s 
     |████████████████████████████████| 623 kB 52.1 MB/s 
     |████████████████████████████████| 456 kB 58.4 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [2]:
!pip install -U spacy-lookups-data

     |████████████████████████████████| 97.3 MB 35 kB/s 


In [3]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.1 MB 19 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
#import libraries
import pandas as pd
import spacy 
from spacy.matcher import Matcher  

In [5]:
#load english language pipeline
nlp = spacy.load('en_core_web_lg')

In [18]:
#Read train data
data = pd.read_excel('list_of_user_stories.xlsx')
data

,User Story
0,"As a Data user, I want to process deletions fo..."
1,"As a UI designer, I want to redesign the Resou..."
2,"As a UI designer, I want to report to the Agen..."
3,"As a UI designer, I want to move on to the rou..."
4,"As a UI designer, I want to move on to the rou..."
...,...
65,"As an Agency user, I want to receive detailed ..."
66,"As a tester, I want to access test features in..."
67,"As a FABS user, I want accurate reflection of ..."
68,"As an Agency user, I want to load all historic..."


In [23]:
#function to return all the matches based on the pattern being passed
def funct_matcher(pattern, i, data):
  
  #instantiate instance of the Matcher
  matcher = Matcher(nlp.vocab) 

  #add pattern to the matcher object
  matcher.add("verb-phrases", [pattern], on_match=None)
  doc = nlp(data['User Story'][i]) 

  #call the matcher to find matches in the doc
  matches = matcher(doc) 
  return matches

In [22]:
def funct_spacy_components_extractor(data):
  #list of columns 
  cols = ['User Story', 'WHO', 'Relationship', 'Entity', 'WHAT', 'WHY']
  data_list = []

  #loop through each user story to extract the main components
  for i in range(0,data['User Story'].count()):
    doc = nlp(data['User Story'][i]) #apply the spacy 
    print(doc)                       #print each of the document

    #extracting the Who component
    chunks_count=0
    for chunk in doc.noun_chunks: # for each noun chunks in each story
      if (chunks_count==0):  #if Condition to fetch the very first noun phrase
          actor = ''

          #put chunk of noun phrase into a doc
          actor_doc = nlp(chunk.text) #apply the spacy package to get the each token of the noun phrase
          tokens_count = len(actor_doc) #count tokens in the noun phrase to apply lemmatisation on the last token only
          t_cnt = 0     #initialise a counter
          for token in actor_doc: #for every token in the noun chunk
            t_cnt = t_cnt + 1
            if token.pos_ != "DET": #exclude determiner from the WHO aspect
              if t_cnt == tokens_count: #if token is the last token in the phrase, apply lemmatisation
                actor = actor + token.lemma_ + ' ' # get the lemma of the token
              else:
                actor = actor + token.text + ' ' #if the token checked is not last token add it to an empty string
            
          print("WHO:", actor)
      chunks_count = chunks_count+1    #get out of the loop and evaluate the next story

    #extracting the What component

    #define pattern to match "I want" or "I want to" phrase, where "to" is optional
    what_pattern = [{'ORTH': 'I'},
            {'LOWER': 'want'},
            {'IS_STOP': True, 'OP': '?'}]

    #call defined function 'funct_matcher' with the specified pattern to return all the matchers for what aspect
    what_matches = funct_matcher(what_pattern, i, data)

    what_start_index = 0 #starting index of what aspect (after excluding 'I want to' phrase)
    orig_what_start_index = 0 #starting index of what aspect (including 'I want to' phrase) for entity extraction 
    
    #loop through all the matchers found
    for match_id, start, end in what_matches:
      orig_what_start_index = start 
      what_start_index = end

    #extracting the Why component
    #define pattern to match "so that" or "so that I" or "so that I can" phrase, where "I" or "I can" is optional
    why_pattern = [{'LOWER': 'so'},
              {'LOWER': 'that'},
              {'ORTH': 'I', 'OP': '?'},
              {'LOWER': 'can', 'OP': '?'}]

  #call defined function 'funct_matcher' with the specified pattern to return all the matchers for why aspect
    why_matches = funct_matcher(why_pattern, i, data)

    what_end_index = 0 #ending index of what aspect
    why_start_index = 0 #starting index of why aspect

    #loop through all the matchers found
    for match_id, start, end in why_matches:
        what_end_index = start
        why_start_index = end

    #extracting the What span from the doc
    what_span = doc[what_start_index:what_end_index-1] #used -1 to remove the comma (before the rationale if any)
    orig_what_span = doc[orig_what_start_index:what_end_index-1] #extract what span including I want [to] phrase
    print("WHAT:", what_span.text)

    what_doc = nlp(orig_what_span.text + ".") #append period to rectify the POS tagging 

    #extracting the relationship from the What aspect
    relationship_list = [] #hold the list of all the relationships identified
    for token in what_doc:
      if (token.pos_ == "VERB"):
        relationship_list.append(token.lemma_)
    print("Relationships:", relationship_list)

    #extracting the entity from the What aspect
    entities_list = [] #hold the list of all the entities identified
    for chunk in what_doc.noun_chunks:
      entities_list.append(chunk.text)   
    print("Entities:", entities_list)

    #fetch second verb and noun phrase to extract the relationship and entity
    relationship = ''
    if (len(relationship_list) >=2):
      relationship = relationship_list[1]

    entity = ''

    if (len(entities_list) >=2):

      #remove the first determiner from extracted entity if any
      entity_doc = nlp(entities_list[1])

      ent_tok_cnt = 0 #count tokens in the entity
      for token in entity_doc:

        if (ent_tok_cnt == 0):

          if token.pos_ != "DET": #exclude determiner from the entity
              entity = entity + token.text + ' '
        else:

          entity = entity + token.text + ' '
        ent_tok_cnt = ent_tok_cnt + 1

    #extracting the Why span from the doc if rationale exist
    if (why_matches != []):

      why_span = doc[why_start_index:]

      print("WHY:", why_span.text)

      data_list.append([data['User Story'][i], actor, relationship, entity, what_span.text, why_span.text])

    else:

      data_list.append([data['User Story'][i], actor, relationship, entity, what_span.text, ''])
    
    print("---------------------------------------------------------------")

  #save the list into a dataframe 
  df = pd.DataFrame(data_list, columns=cols)

  return df



In [24]:
#call spacy components extractor function to return the extracted results
spacy_df = funct_spacy_components_extractor(train_data)
spacy_df.to_csv("list_of_user_stories_extracted_Spacy.csv")
spacy_df

As a Data user, I want to process deletions for 12-19-2017 .
WHO: Data user 
WHAT: process deletions for 12-19-2017
Relationships: ['want', 'process']
Entities: ['I', 'deletions']
---------------------------------------------------------------
As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.
WHO: UI designer 
WHAT: redesign the Resources page
Relationships: ['want', 'redesign']
Entities: ['I', 'the Resources page']
WHY: it matches the new Broker design styles.
---------------------------------------------------------------
As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.
WHO: UI designer 
WHAT: report to the Agencies about user testing
Relationships: ['want', 'report']
Entities: ['I', 'the Agencies', 'user testing']
WHY: they are aware of their contributions to making Broker a better UX.
----------------------------------------------

,User Story,WHO,Relationship,Entity,WHAT,WHY
0,"As a Data user, I want to process deletions fo...",Data user,process,deletions,process deletions for 12-19-2017,
1,"As a UI designer, I want to redesign the Resou...",UI designer,redesign,Resources page,redesign the Resources page,it matches the new Broker design styles.
2,"As a UI designer, I want to report to the Agen...",UI designer,report,Agencies,report to the Agencies about user testing,they are aware of their contributions to makin...
3,"As a UI designer, I want to move on to the rou...",UI designer,move,DABS or FABS landing page edits,move on to the round 2 of DABS or FABS landing...,get approvals from leadership.
4,"As a UI designer, I want to move on to the rou...",UI designer,move,Homepage edits,move on to the round 2 of Homepage edits,get approvals from leadership.
...,...,...,...,...,...,...
65,"As an Agency user, I want to receive detailed ...",Agency user,receive,detailed file - level error,receive detailed file-level error when I uploa...,
66,"As a tester, I want to access test features in...",tester,access,test features,access test features in environments other tha...,test any nonProd feature in any environment.
67,"As a FABS user, I want accurate reflection of ...",FABS user,access,accurate reflection,accurate reflection of frontend URLs in the pa...,am not confused.
68,"As an Agency user, I want to load all historic...",Agency user,load,historical Financial Assistance data,load all historical Financial Assistance data ...,
